# Azure Cloud Cost Optimization Advisor
This notebook analyzes Azure Cost Management data to identify savings opportunities with GenAI-powered recommendations.

In [ ]:
# Azure-specific dependencies
!pip install pandas matplotlib opencensus

In [ ]:
# Upload Azure billing file
from google.colab import files
uploaded = files.upload()
file_name = next(iter(uploaded))

In [ ]:
# Process Azure billing data
import pandas as pd

df = pd.read_csv(file_name, low_memory=False)

# Azure-specific columns
required_columns = ['ServiceName', 'Cost', 'Date']
if not all(col in df.columns for col in required_columns):
    df.columns = df.columns.str.lower()
    df = df.rename(columns={
        'servicename': 'ServiceName',
        'costinbillingcurrency': 'Cost',
        'date': 'Date'
    })

print("Top Azure Services by Cost:")
print(df.groupby('ServiceName')['Cost'].sum().sort_values(ascending=False).head(10))

In [ ]:
# Azure Cost Visualization
import matplotlib.pyplot as plt

if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    cost_by_day = df.groupby('Date')['Cost'].sum()
    
    plt.figure(figsize=(12, 6))
    cost_by_day.plot(kind='line', marker='o', color='#007FFF')
    plt.title('Daily Azure Spend', pad=20)
    plt.ylabel('Cost ($)')
    plt.xlabel('Date')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Azure-specific cost optimization
def azure_recommendations(service, cost):
    recommendations = {
        'Virtual Machines': [
            "Reserved Instances can save up to 72% compared to pay-as-you-go",
            "Consider Azure Spot VMs for non-critical workloads (up to 90% savings)",
            "Right-size underutilized VMs using Azure Advisor"
        ],
        'Storage': [
            "Move cold data to Cool/Archive storage tiers",
            "Enable lifecycle management policies",
            "Review blob storage access tiers"
        ],
        'SQL Database': [
            "Consider serverless compute tier for intermittent workloads",
            "Implement auto-pause during off-hours",
            "Review DTU/vCore provisioning"
        ],
        'App Service': [
            "Switch to Premium v3 plans for better price-performance",
            "Implement auto-scaling rules",
            "Review always-on setting for non-production apps"
        ]
    }
    
    return recommendations.get(service, ["Review service configuration in Azure Cost Management"])

In [ ]:
# Generate Azure optimization report
top_services = df.groupby('ServiceName')['Cost'].sum().nlargest(5)

print("\nAzure Cost Optimization Recommendations:")
for service, cost in top_services.items():
    print(f"\n🔹 {service} (${cost:.2f}):")
    for rec in azure_recommendations(service, cost):
        print(f" • {rec}")